In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import constants as ct

# Limpeza dos dados

Dataset Original

In [2]:
df_gripe_19 = pd.read_csv(ct.PATH_GRIPE_19, low_memory=False, sep=';')
df_gripe_20 = pd.read_csv(ct.PATH_GRIPE_20, low_memory=False, sep=';')

Concatenamos os datasets para gerar um único dataset de casos

In [3]:
df_gripe = pd.concat([df_gripe_19,df_gripe_20],axis=0)

# Selected Columns

In [4]:
COL_DATA_NOTIF = 'Data da notificação'
COL_IDADE = 'Idade'
COL_SEXO = 'Sexo'
COL_DATA_ENTRADA = 'Data de internação'
COL_DATA_ENTRADA_UTI = 'Data da entrada na UTI'
COL_DATA_SAIDA_UTI = 'Data de saída da UTI'
COL_DATA_SAIDA = 'Data da alta ou óbito'
COL_CLASSIF = 'Classificação final do caso'
COL_BAIRRO = 'Bairro'
COL_RESOLUCAO = 'Evolução do caso'
COL_CNES_INT = 'CNES da unidade de saúde de internação'

Removemos linhas sem data de internação

In [5]:
df_gripe = df_gripe[~df_gripe[COL_DATA_ENTRADA].isna()]

Removemos uma data fora do range de datetime

In [6]:
df_gripe = df_gripe[df_gripe[COL_DATA_ENTRADA] != '28/04/7202'].copy()

Transformamos datas em datetime

In [7]:
df_gripe[COL_DATA_ENTRADA] = pd.to_datetime(df_gripe[COL_DATA_ENTRADA], format='%d/%m/%Y')

Remove internações futuras

In [8]:
df_gripe = df_gripe[df_gripe[COL_DATA_ENTRADA] < datetime.now()].copy()

Retornamos o datetime de entrada ao formato original

In [9]:
df_gripe[COL_DATA_ENTRADA] = df_gripe[COL_DATA_ENTRADA].dt.strftime("%d/%m/%Y")

Criamos dataset apenas com as colunas de interesse para computar a previsão de ocupação

In [10]:
min_cols = [
    COL_DATA_NOTIF, 
    COL_DATA_ENTRADA, COL_DATA_SAIDA, 
    COL_DATA_ENTRADA_UTI, COL_DATA_SAIDA_UTI,
    COL_CNES_INT, 
    COL_IDADE, COL_SEXO,COL_BAIRRO,
    COL_CLASSIF, COL_RESOLUCAO]
df_gripe_min = df_gripe[min_cols].copy()

Quantidades de Nans em cada coluna

In [11]:
df_gripe_min.isna().sum(axis=0)

Data da notificação                           0
Data de internação                            0
Data da alta ou óbito                     10647
Data da entrada na UTI                    10427
Data de saída da UTI                      16599
CNES da unidade de saúde de internação     1694
Idade                                         0
Sexo                                          0
Bairro                                      643
Classificação final do caso                8593
Evolução do caso                           9986
dtype: int64

Convertemos o código de CNES para inteiros para facilitar o merge com outros datasets. Preenchemos Nan com 0s

In [12]:
df_gripe_min[COL_CNES_INT] = df_gripe_min[COL_CNES_INT].fillna(0)
df_gripe_min[COL_CNES_INT] = df_gripe_min[COL_CNES_INT].astype(int)

Salvamos o dataset filtrado

In [13]:
df_gripe_min.to_csv(ct.PATH_GRIPE_MIN, index=False)